In [ ]:
#pip install gmplot ortools

In [ ]:
import torch

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances

In [ ]:
from datetime import datetime
import time
import gmplot # построение HTML для визуализации
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from numpy import mean

In [ ]:
#project_path = '/mnt/c/Python_Projects/ITMO_Bootcamp'
points = pd.read_csv('refinded_geo_dataset.csv')

In [ ]:
cb_coords_df = {'LATITUDE': [54.952542],
                'LONGITUDE': [73.330175]
                }
cb_coords_df = pd.DataFrame(cb_coords_df)
LATLON = ['LATITUDE', 'LONGITUDE']

In [ ]:

def get_distance_matrix(x, y):
    """
    Используется в классе Points для подсчета расстояний между всеми заказами
    и в классе CourierMapping для подсчета расстояний между центроидами курьеров (метры?)
    :param x: Вектор координат х
    :param y: Вектор координат у
    :return: Квадратная матрица расстояний (n x n) с попарными расстояниями
    """
    C = torch.stack((x, y), dim=1)
    R = torch.sum(C * C, dim=1, keepdim=True)
    R_T = torch.transpose(R, dim0=0, dim1=1)
    C_T = torch.transpose(C, dim0=0, dim1=1)
    CmC_T = torch.mm(C, C_T)
    D2 = R - 2 * CmC_T + R_T
    D = torch.sqrt(torch.abs(D2))
    return D.float()


def degrees_to_radians(arr):
    return np.pi * arr / 180

In [ ]:
points = points[:20].append(cb_coords_df)
points_cords = [(degrees_to_radians(lat), degrees_to_radians(lon)) for lat, lon in
                points[LATLON].values.tolist()]
distance_matrix = haversine_distances(points_cords) * (6371000 / 1000)  # коэффицент для перевода в км#

<ipython-input-8-dfc02a54a651>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  points = points[:20].append(cb_coords_df)


In [ ]:
distance_matrix


# class RoutePoint

In [ ]:
class RoutePoint:
    """Simple point of route"""

    def __init__(self, index, deadline, lat: float = None, lon: float = None):

        if not isinstance(lat, float):
            raise TypeError("required float latitude value")
        if not isinstance(lon, float):
            raise TypeError("required float longitude value")

        if deadline:
            self.deadline = datetime.strptime(deadline, "%Y-%m-%d %H:%M:%S.%f")
            self.POSIX = time.mktime(self.deadline.timetuple())
        else:
            self.deadline = None
            self.POSIX = None

        self.start_lat_lon = (lat, lon)
        self.rosm_id_point = index
        self.pos = None
        self.vehicle_id = None


class RoutePlannigTask:
    """class for route planning"""

    def __init__(self, num_vehicles:int, mode="dist", verbose=True):
        self.destinations = None
        self.origins = None
        self.places = None
        self.start_place = None
        self.num_vehicles = num_vehicles
        self.mode = mode
        self.matrix = None
        self.verbose = verbose

        self.distance_matrix = None

        self.dataset = None
        self.solution = None
        self.manager = None
        self.routing = None
        self.vert_path = None
        self.fig = None
        self.last_route_point_index = None

    def set_params(self, start_place, origins, matrix, last_route_point_index=None):
        """
        This function allow set required parameters for computing

        :param start_place: list of start place
        :param origins: list of delivety points
        :param matrix:
        :param last_route_point_index:
        :return:
        """
        if not isinstance(start_place, list):
            raise TypeError(
                f"required list type as starting point argument but"
                f" {type(start_place)} received"
            )
        if not isinstance(origins, list):
            raise TypeError(
                f"required list type as origins point argument but"
                f" {type(origins)} received"
            )

        if not isinstance(last_route_point_index, int) and (
                last_route_point_index is not None
        ):
            raise TypeError(
                f"required int of last point index or None"
                f" {type(last_route_point_index)} received"
            )

        if isinstance(matrix, np.ndarray):
            matrix = matrix.tolist()

        if not isinstance(matrix, list):
            raise TypeError(
                f"required distance matrix as list of lists argument but"
                f" {type(origins)} received"
            )

        # set parameters
        self.places = origins
        self.origins = start_place + origins
        self.destinations = start_place + origins

        self.start_place = start_place
        self.matrix = matrix

        if (last_route_point_index == 0) or (last_route_point_index is None):
            self.last_route_point_index = None
        else:
            self.last_route_point_index = last_route_point_index

    def _create_data_model(self):
        """Stores the data for the problem."""

        self.dataset = {}

        if self.mode == "dist":
            self.dataset["distance_matrix"] = self.matrix

        else:
            raise ValueError("Unknown value")

        self.dataset["num_vehicles"] = self.num_vehicles
        self.dataset["start"] = [0]
        if (self.last_route_point_index is None) or (self.last_route_point_index == 0):
            self.dataset["end"] = [0]
        else:

            self.dataset["end"] = [self.last_route_point_index]

    def print_solution(self, data, manager, routing, solution):
        """Prints solution on console."""

        if self.verbose:
            print(f"Objective: {solution.ObjectiveValue()}")
        max_route_distance = 0
        pos = 0
        for vehicle_id in range(data["num_vehicles"]):
            index = routing.Start(vehicle_id)
            plan_output = "Route for vehicle {}:\n".format(vehicle_id)
            route_distance = 0
            while not routing.IsEnd(index):
                self.origins[manager.IndexToNode(index)].pos = pos
                self.origins[manager.IndexToNode(index)].vehicle_id = vehicle_id

                plan_output += " {} -> ".format(manager.IndexToNode(index))
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id
                )
                pos += 1

            if self.last_route_point_index is not None:

                for i in range(len(self.origins)):
                    if self.origins[i].rosm_id_point == self.last_route_point_index:
                        self.origins[self.last_route_point_index].pos = 999
                        self.origins[
                            self.last_route_point_index
                        ].vehicle_id = vehicle_id

            plan_output += "{}\n".format(manager.IndexToNode(index))
            plan_output += "Distance of the route: {}m\n".format(route_distance)
            if self.verbose:
                print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)
        if self.verbose:
            print("Maximum of the route distances: {}m".format(max_route_distance))

    def get_optimal_path(self):

        """
        Thus function can obtain an optimal path if it exists.
        """
        self._create_data_model()

        self.manager = pywrapcp.RoutingIndexManager(
            len(self.dataset["distance_matrix"]),
            self.dataset["num_vehicles"],
            self.dataset["start"],
            self.dataset["end"],
        )

        self.routing = pywrapcp.RoutingModel(self.manager)

        # Create and register a transit callback.
        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.dataset["distance_matrix"][from_node][to_node]

        transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Distance constraint.
        dimension_name = "Distance"
        self.routing.AddDimension(
            transit_callback_index, 0, 10000000, True, dimension_name  # no slack
        )
        distance_dimension = self.routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )

        # Solve the problem.
        solution = self.routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        if solution:
            self.print_solution(self.dataset, self.manager, self.routing, solution)
        else:
            raise RuntimeError("Solution not found. Check input parameters")

        return self.origins

    def plot_points(self, points, filename="result.html"):
        """
        Plot a route on a map.
        param points:  Array of points
        :param filename:  Filename to save an output
        :return:
        """

        # extract latitudes
        latitude_list = [
            i.start_lat_lon[0]
            for i in sorted(points, key=lambda p: p.pos, reverse=False)
        ]
        # extract longitudes
        longitude_list = [
            i.start_lat_lon[1]
            for i in sorted(points, key=lambda p: p.pos, reverse=False)
        ]

        # extract labels
        label_list = [
            i.rosm_id_point for i in sorted(points, key=lambda p: p.pos, reverse=False)
        ]
        color_list = ["#87CEEB"] * len(points)
        color_list[0] = "#98FB98"

        if (self.last_route_point_index is not None) or (
                self.last_route_point_index != 0
        ):
            color_list[-1] = "#FA8072"

        assert len(latitude_list) == len(longitude_list) == len(color_list)

        gmap3 = gmplot.GoogleMapPlotter(mean(latitude_list), mean(longitude_list), 13)
        gmap3.scatter(
            latitude_list,
            longitude_list,
            size=30,
            marker=True,
            label=label_list,
            color=color_list,
        )
        gmap3.plot(latitude_list, longitude_list, "cornflowerblue", edge_width=2.5)

        if (self.last_route_point_index is None) or (self.last_route_point_index == 0):
            gmap3.plot(
                [latitude_list[-1], latitude_list[0]],
                [longitude_list[-1], longitude_list[0]],
                "#D27D2D",
                edge_width=2.5,
            )
        gmap3.draw(filename)


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances


start_point = []
points = []

coords = [(37.769901, -122.498331),
          (37.768645, -122.475328),
          (37.769867, -122.466102),
          (37.767187, -122.467496),
          (37.770104, -122.470436)]

dm_for_courier = [[0, 1, 2, 3, 4],   # distance_matrix
                       [5, 0, 6, 7, 8],
                       [9, 10, 0, 11, 12],
                       [13, 14, 15, 0, 16],
                       [17, 18, 19, 20, 0],
                       ]

for index, i in enumerate(coords):
    if index == 0:
        point = RoutePoint(index=0, deadline=None, lat=i[0], lon=i[1])
        start_point.append(point)
    elif index == 1:
        point = RoutePoint(index=0, deadline=None, lat=i[0], lon=i[1])
        start_point.append(point)
    else:
        point = RoutePoint(index=index, deadline=None, lat=i[0], lon=i[1])
        points.append(point)


In [ ]:
RoutePoint

__main__.RoutePoint

In [ ]:
start_point

In [ ]:
points

In [ ]:
dm_for_courier

[[0, 1, 2, 3, 4],
 [5, 0, 6, 7, 8],
 [9, 10, 0, 11, 12],
 [13, 14, 15, 0, 16],
 [17, 18, 19, 20, 0]]

In [ ]:
route = RoutePlannigTask(num_vehicles=2, verbose=False)



In [ ]:
route

In [ ]:
route.set_params(
    start_point, points, dm_for_courier)


In [ ]:
route_points = route.get_optimal_path()


In [ ]:
# Get optimal path for the first starting point.
route.set_params([start_point[0]], points, dm_for_courier)
path1 = route.get_optimal_path()
route.plot_points(points, "result1.html")

In [ ]:
# Get optimal path for the second starting point.
route.set_params(start_point[1], points, dm_for_courier)
path2 = route.get_optimal_path()
route.plot_points(points, "result2.html")

In [ ]:
route.plot_points(points, "result.html")

In [ ]:
points.fillna(-1 , inplace=True)
points['idx'] = points['Unnamed: 0'].apply(int)

In [ ]:
points = points[LATLON + ['idx']]

In [ ]:
points

,LATITUDE,LONGITUDE,idx
0,54.989735,73.284150,0
1,55.016132,73.195114,1
2,55.037613,73.268270,2
3,55.030422,73.330430,3
4,55.037056,73.274660,4
5,55.016132,73.195114,5
6,55.054520,73.220350,6
7,55.021797,73.431580,7
8,55.054520,73.220350,8
9,55.031464,73.267290,9


In [ ]:
cb_coords_df.loc[0, LATLON[0]]

54.952542

In [ ]:
start_point = [RoutePoint(index=-1,
                          deadline=None,
                          lat=cb_coords_df.loc[0, LATLON[0]],
                          lon=cb_coords_df.loc[0, LATLON[1]])]
start_point

In [ ]:
points[["idx"] + LATLON][:-1].values.tolist()

In [ ]:

## Созданиие маршрутов
print("Создаем маршруты")
dm = distance_matrix # distance_matrix
# dm = points_obj.haversine_distance_matrix
# кб / index = -1, Влияет на метод points_obj.predict_to_orders()
start_point = [RoutePoint(index=-1,
                          deadline=None,
                          lat=cb_coords_df.loc[0, LATLON[0]],
                          lon=cb_coords_df.loc[0, LATLON[1]])]

courier_task = []
df = points

for index, lat, lon in df[["idx"] + LATLON][:-1].values.tolist():
    point = RoutePoint(index=int(index), deadline=None, lat=lat, lon=lon)
    print(int(index), lat, lon)
    print(point)
    courier_task.append(point)

route = RoutePlannigTask(num_vehicles=9, verbose=True)

route.set_params(start_point*9,
                  courier_task,
                  dm.tolist(),
                 #last_route_point_index=11
                )

courier_task = route.get_optimal_path()

route.plot_points(courier_task, "{}.html".format("test"))
